<a href="https://colab.research.google.com/github/akashsharma-2002/text-classification-on-a-given-dataset-of-news-articles/blob/main/text_classification_on_a_given_dataset_of_news_articles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive



TEXT CLASSIFICATION ON A GIVEN DATASET OF NEWS ARTICLE

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from transformers import BertTokenizer, TFBertForSequenceClassification
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from transformers import AdamW

INSTALLING TRANSFORMER FOR USING BERT

In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 52.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 89.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 68.0 MB/s eta 0:00:00


DATASET USED HERE IS FROM BBC NEWS

In [6]:
df = pd.read_csv('/content/BBC News Sample Solution.csv')

PRINTING THE WHOLE COLUMN OF DATASET

In [8]:
df

,ArticleId,Category
0,1018,sport
1,1319,tech
2,1138,business
3,459,entertainment
4,1020,politics
...,...,...
730,1923,sport
731,373,tech
732,1704,business
733,206,entertainment


USING ARTICLEID AS TEXT AND LABEL AS CATEGORY

In [20]:
text = df['ArticleId'].astype(str)
labels = df['Category']

NOW PERFORMING TOKENIZATION ON TEXT

In [21]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
inputs = tokenizer(text.tolist(), return_tensors='tf', truncation=True, padding=True, max_length=512)

ENCODING THE LABEL

In [22]:
encoder = LabelEncoder()
labels = encoder.fit_transform(labels)
labels = tf.keras.utils.to_categorical(labels)

FOR SPLITTING THE DATA INPUT CANNOT BE DIRECTLY ACCESSED THUS CONVERTING INPUT TO NUMPY ARRAY

In [18]:
import numpy as np

In [23]:
input_ids = inputs['input_ids'].numpy()
attention_mask = inputs['attention_mask'].numpy()

In [19]:
input_ids = np.array(inputs['input_ids'])

SPLITTING THE DATA INTO TRAINING AND TESTING DATASET

In [24]:
train_input_ids, test_input_ids, train_labels, test_labels = train_test_split(input_ids, labels, test_size=0.2, random_state=42)
train_attention_mask, test_attention_mask, _, _ = train_test_split(attention_mask, labels, test_size=0.2, random_state=42)

INPUT AND LABEL INTO TENSORFLIOW DATASET

In [25]:
train_inputs = {'input_ids': train_input_ids, 'attention_mask': train_attention_mask}
test_inputs = {'input_ids': test_input_ids, 'attention_mask': test_attention_mask}

In [26]:
train_tf_dataset = tf.data.Dataset.from_tensor_slices((train_inputs, train_labels)).batch(32)
test_tf_dataset = tf.data.Dataset.from_tensor_slices((test_inputs, test_labels)).batch(32)

DEFINING THE MODEL

In [27]:
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(encoder.classes_))

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


COMPILING THE MODEL

In [28]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

TRAINING THE MODEL WITH 12 EPOCHS

In [35]:
history = model.fit(train_tf_dataset, epochs=12, validation_data=test_tf_dataset)

Epoch 1/12
19/19 [==============================] - 68s 3s/step - loss: 8.6987 - accuracy: 0.1990 - val_loss: 6.7981 - val_accuracy: 0.2041
Epoch 2/12
19/19 [==============================] - 62s 3s/step - loss: 7.9517 - accuracy: 0.2024 - val_loss: 9.3200 - val_accuracy: 0.2041
Epoch 3/12
19/19 [==============================] - 64s 3s/step - loss: 8.8564 - accuracy: 0.2024 - val_loss: 9.3200 - val_accuracy: 0.2041
Epoch 4/12
19/19 [==============================] - 59s 3s/step - loss: 9.1566 - accuracy: 0.1905 - val_loss: 9.3200 - val_accuracy: 0.2041
Epoch 5/12
19/19 [==============================] - 59s 3s/step - loss: 8.6084 - accuracy: 0.1973 - val_loss: 9.3200 - val_accuracy: 0.2041
Epoch 6/12
19/19 [==============================] - 59s 3s/step - loss: 8.4981 - accuracy: 0.2041 - val_loss: 9.3200 - val_accuracy: 0.2041
Epoch 7/12
19/19 [==============================] - 62s 3s/step - loss: 8.2241 - accuracy: 0.1905 - val_loss: 9.3200 - val_accuracy: 0.2041
Epoch 8/12
19/19 [==

In [36]:
from sklearn.metrics import classification_report
import numpy as np

PREDICTING THE MODELS ACCURACY, PRECISION AND F1-SCORE



In [37]:
predictions = model.predict(test_inputs)['logits']

5/5 [==============================] - 2s 450ms/step


In [38]:
predictions = np.argmax(predictions, axis=1)

In [39]:
true_labels = np.argmax(test_labels, axis=1)

In [40]:
report = classification_report(true_labels, predictions, target_names=encoder.classes_)
print(report)


               precision    recall  f1-score   support

     business       0.00      0.00      0.00        15
entertainment       0.00      0.00      0.00        31
     politics       0.20      1.00      0.34        30
        sport       0.00      0.00      0.00        32
         tech       0.00      0.00      0.00        39

     accuracy                           0.20       147
    macro avg       0.04      0.20      0.07       147
 weighted avg       0.04      0.20      0.07       147



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


SAAVING THE MODEL

In [41]:
model.save_pretrained('./saved_model')